[adaptado de [Programa de cursos integrados Aprendizado de máquina](https://www.coursera.org/specializations/machine-learning-introduction) de [Andrew Ng](https://www.coursera.org/instructor/andrewng)  ([Stanford University](http://online.stanford.edu/), [DeepLearning.AI](https://www.deeplearning.ai/) ) ]

In [ ]:
# Baixar arquivos adicionais para o laboratório
!wget https://github.com/fabiobento/dnn-course-2024-1/raw/main/00_course_folder/nn_adv/class_03/Laborat%C3%B3rios/lab_utils_ml_adv_week_3.zip
      
!unzip -n -q lab_utils_ml_adv_week_3.zip

In [ ]:
# Testar se estamos no Google Colab
# Necessário para ativar widgets
try:
  import google.colab
  IN_COLAB = True
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  IN_COLAB = False

# Diagnosticando o viés e a variância

No laboratório opcional anterior, você viu como avaliar o desempenho de um algoritmo de aprendizagem medindo o erro de treinamento e validação cruzada.

Com esses valores, é possível quantificar o desempenho de um modelo, o que ajuda a tomar uma decisão sobre qual deles usar para um determinado aplicativo.

Neste laboratório, você se baseará nesse processo e explorará algumas dicas para melhorar o desempenho dos seus modelos.

Como se vê, os erros de treinamento e validação cruzada podem lhe dizer o que tentar em seguida para melhorar seus modelos. Especificamente, eles mostrarão se você tem um problema de alto viés  (subajuste - _underfitting_) ou alta variância (sobreajuste -_overfitting_). O slide da aula mostrado abaixo apresenta um exemplo:

<img src='images/C2_W3_BiasVariance.png' width=75%>

A figura mais à esquerda mostra um problema de alta tendência em que o modelo não está capturando os padrões nos dados de treinamento. Como resultado, você terá um erro alto de treinamento e validação cruzada.

A figura mais à direita, por outro lado, mostra um problema de alta variância em que o modelo supera o conjunto de treinamento. Portanto, embora tenha um erro de treinamento baixo, ele terá um desempenho ruim em novos exemplos. Isso é indicado por um alto erro de validação cruzada.

O modelo ideal seria a figura do meio, em que ele aprende com sucesso a partir do conjunto de treinamento e também generaliza bem para dados não vistos. As palestras deram algumas dicas sobre o que fazer em seguida para obter esse modelo "ideal". 

Para corrigir um problema de _underfitting_, você pode:
* tentar adicionar recursos polinomiais
* tentar obter recursos adicionais
* tentar diminuir o parâmetro de regularização

Para corrigir um problema de _overfitting_, você pode:
* tentar aumentar o parâmetro de regularização
* Experimentar conjuntos menores de recursos
* obter mais exemplos de treinamento

Você experimentará todas essas dicas neste laboratório. Vamos começar!

## Estabelecendo o nível de desempenho da linha de base

Antes de diagnosticar um modelo com alto viés ou alta variância, geralmente é útil ter uma ideia do nível de erro ao qual você pode tipicamente chegar. Conforme mencionado na aula, você pode usar qualquer um dos seguintes itens para definir um nível de desempenho de linha de base.

* desempenho em nível humano
* desempenho do algoritmo concorrente
* suposição baseada na experiência

Os dados do mundo real podem ser muito ruidosos e muitas vezes é inviável chegar a 0% de erro. Por exemplo, você pode achar que tem um problema de _underfitting_ porque está obtendo 10% de erro no treinamento e 15% de erro na validação cruzada em uma aplicação de visão computacional.

No entanto, mais tarde você descobriu que nem mesmo os humanos conseguem ter um desempenho melhor do que 10% de erro. Se você considerar esse o nível de linha de base, terá um problema de _overfitting_ porque priorizou a minimização da lacuna entre a validação cruzada e o erro de treinamento.

Com isso em mente, vamos começar a explorar as técnicas para resolver esses problemas.

## Importações e configuração do laboratório

Com exceção de alguns [regressores lineares](https://scikit-learn.org/stable/modules/classes.html#classical-linear-regressors) do scikit-learn, todas as outras funções usadas neste laboratório são encontradas no arquivo `utils.py` fora deste caderno.

Na maioria das vezes, você usará o mesmo código do último laboratório, portanto, não é necessário ver cada linha aqui novamente. Ele contém principalmente funções para dividir os dados, bem como funções que fazem loop em uma lista de parâmetros (por exemplo, grau do polinômio, parâmetro de regularização) e plotam o erro de treinamento e validação cruzada para cada um deles. Sinta-se à vontade para explorar o código no referido arquivo para ver a implementação.

In [ ]:
# para criar modelos de regressão linear
from sklearn.linear_model import LinearRegression, Ridge

# Importar funções utilitárias do laboratório em utils.py
import utils 

## Correção de viés elevado

Primeiro, você verá o que deve ser tentado quando o modelo estiver com ajuste _undefitting_. 

Em outras palavras, quando o erro de treinamento é muito pior do que o nível de desempenho da linha de base.

### Tente adicionar recursos polinomiais

Você já viu isso no laboratório anterior:
> A adição de recursos polinomiais pode ajudar o modelo a aprender padrões mais complexos nos dados.

Aqui está novamente um exemplo de um gráfico que mostra como os erros de treinamento e validação cruzada mudam à medida que você adiciona mais recursos polinomiais.

Você usará um conjunto de dados sintético para um problema de regressão com um recurso e um alvo. Além disso, você também definirá um desempenho de linha de base arbitrário e o incluirá no gráfico.

In [ ]:
# Dividir o conjunto de dados em train, cv e test
x_train, y_train, x_cv, y_cv, x_test, y_test = utils.prepare_dataset('data/c2w3_lab2_data1.csv')

print(f"O formato do conjunto de treino (entrada) é: {x_train.shape}")
print(f"O formato do conjunto de treino (alvo) é: {y_train.shape}\n")
print(f"O formato do conjunto de validação cruzada (entrada) é: {x_cv.shape}")
print(f"O formato do conjunto de validação cruzada (target) é: {y_cv.shape}\n")

# Visualizar as primeiras 5 linhas
print(f"As primeiras 5 linhas das entradas de treinamento (1 característica):\n {x_train[:5]}\n")

# Instanciar a classe do modelo de regressão
model = LinearRegression()

# Treinar e plotar modelos de regressão polinomial
utils.train_plot_poly(model, x_train, y_train, x_cv, y_cv, max_degree=10, baseline=400)

Como você pode ver, quanto mais recursos polinomiais forem adicionados, melhor será o ajuste do modelo aos dados de treinamento.

Neste exemplo, o desempenho foi até melhor que o da linha de base. Neste ponto, você pode dizer que os modelos com grau maior que 4 são de baixa polarização porque têm desempenho próximo ou melhor que a linha de base.

No entanto, se a linha de base for definida como inferior (por exemplo, você consultou um especialista sobre o erro aceitável), os modelos ainda serão considerados de alta polarização. Você pode então tentar outros métodos para melhorar isso.

In [ ]:
# Treine e desenhe modelos de regressão polinomial. O viés é definido abaixo.
utils.train_plot_poly(model, x_train, y_train, x_cv, y_cv, max_degree=10, baseline=250)

### Tente obter recursos adicionais

Outra coisa que você pode tentar é adquirir outros recursos.

Digamos que, depois de obter os resultados acima, você tenha decidido lançar outra campanha de coleta de dados que capture outro recurso.

Seu conjunto de dados agora terá duas colunas para os recursos de entrada, conforme mostrado abaixo.

In [ ]:
x_train, y_train, x_cv, y_cv, x_test, y_test = utils.prepare_dataset('data/c2w3_lab2_data2.csv')

print(f"o formato do conjunto de treino (entrada) é: {x_train.shape}")
print(f"o formato do conjunto de treino (alvo) é: {y_train.shape}\n")
print(f"o formato do conjunto de validação cruzada (entrada) é: {x_cv.shape}")
print(f"o formato do conjunto de validação cruzada (alvo) é: {y_cv.shape}\n")

# Preview the first 5 rows
print(f"As primeiras 5 linhas das entradas de treinamento (2 recursos):\n {x_train[:5]}\n")

Agora veja o que isso faz com o mesmo processo de treinamento de antes. Você notará que o erro de treinamento agora está mais próximo (ou até melhor) do que a linha de base.

In [ ]:
# Instanciar a classe do modelo
model = LinearRegression()

# Treinar e plotar modelos de regressão polinomial. O conjunto de dados usado tem dois recursos.
utils.train_plot_poly(model, x_train, y_train, x_cv, y_cv, max_degree=6, baseline=250)

### Tente diminuir o parâmetro de regularização

Neste ponto, talvez você queira introduzir a regularização para evitar o _overfitting_.

Um aspecto a ser observado é que você pode fazer com que seus modelos fiquem subajustados se definir o parâmetro de regularização muito alto.

A célula abaixo treina um modelo polinomial de 4º grau usando a classe [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge) que permite definir um parâmetro de regularização (ou seja, lambda ou $\lambda$). Você tentará vários valores e comparará os resultados.

In [ ]:
# Definir lambdas para plotar
reg_params = [10, 5, 2, 1, 0.5, 0.2, 0.1]

# Defina o grau do polinômio e treine para cada valor de lambda
utils.train_plot_reg_params(reg_params, x_train, y_train, x_cv, y_cv, degree= 4, baseline=250)

O gráfico resultante mostra um $\lambda$ inicial de `10` e, como você pode ver, o erro de treinamento é pior do que a linha de base nesse ponto.

Isso implica que ele está colocando uma penalidade enorme nos parâmetros `w` e isso impede que o modelo aprenda padrões mais complexos em seus dados.

À medida que você diminui $\lambda$, o modelo afrouxa essa restrição e o erro de treinamento consegue se aproximar do desempenho da linha de base.

## Como corrigir a alta variância

Agora, você verá algumas ações que podem ser tentadas quando o seu modelo tiver se ajustado demais ao conjunto de treinamento.

O objetivo principal é ter um modelo que generalize bem para novos exemplos, de modo que você queira minimizar o erro de validação cruzada.

### Tente aumentar o parâmetro de regularização

Em contraste com o último exercício acima, a definição de um valor muito pequeno do parâmetro de regularização manterá o modelo com viés baixo, mas poderá não contribuir muito para melhorar a variância.

Conforme mostrado abaixo, você pode melhorar o erro de validação cruzada aumentando o valor de $\lambda$.

In [ ]:
# Definir lambdas para plotar
reg_params = [0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1]

# Defina o grau do polinômio e treine para cada valor de lambda
utils.train_plot_reg_params(reg_params, x_train, y_train, x_cv, y_cv, degree= 4, baseline=250)

### Experimente conjuntos menores de recursos

Você já viu no último laboratório que ter muitos termos polinomiais pode resultar em ajuste excessivo. Você pode reduzir o número desses termos e ver onde obtém o melhor equilíbrio de erros de treinamento e validação cruzada.

Outro cenário em que a redução do número de recursos seria útil é quando você tem recursos irrelevantes em seus dados. Por exemplo, os IDs de pacientes fornecidos pelos hospitais não ajudarão no diagnóstico de um tumor, portanto, você deve removê-los dos dados de treinamento. 

Para ilustrar como a remoção de recursos pode melhorar o desempenho, você fará a regressão polinomial para dois conjuntos de dados: os mesmos dados que usou acima (2 recursos) e outro com uma coluna de ID aleatória (3 recursos). Você pode visualizá-los usando a célula abaixo. Observe que duas colunas são idênticas e uma terceira é adicionada para incluir números aleatórios.

In [ ]:
# Preparar o conjunto de dados com o recurso randomID
x_train, y_train, x_cv, y_cv, x_test, y_test = utils.prepare_dataset('data/c2w3_lab2_data2.csv')

# Visualizar as primeiras 5 linhas
print(f"As primeiras 5 linhas do conjunto de treinamento com 2 recursos:\n {x_train[:5]}\n")

# Preparar o conjunto de dados com o recurso randomID
x_train, y_train, x_cv, y_cv, x_test, y_test = utils.prepare_dataset('data/c2w3_lab2_data3.csv')

# Preview the first 5 rows
print(f"As primeiras 5 linhas do conjunto de treinamento com 3 recursos (a 1ª coluna é um ID aleatório):\n {x_train[:5]}\n")

Agora você treinará os modelos e plotará os resultados.

As linhas sólidas no gráfico mostram os erros para os dados com 2 recursos, enquanto as linhas pontilhadas mostram os erros para o conjunto de dados com 3 recursos.

Como você pode ver, o modelo com 3 recursos tem um erro de validação cruzada maior, especialmente quando você introduz mais termos polinomiais. Isso ocorre porque o modelo também está tentando aprender com as IDs aleatórias, embora elas não tenham nada a ver com o alvo. 

Outra maneira de ver isso é observar os pontos no grau = 4. Você perceberá que, embora o *erro de treinamento* seja menor com 3 recursos, a *distância entre o erro de treinamento e o erro de validação cruzada* é muito maior do que quando você usa apenas 2 recursos. Isso também deve avisá-lo de que o modelo está se ajustando demais.

In [ ]:
# Definir o modelo
model = LinearRegression()

# Definir propriedades dos 2 conjuntos de dados
file1 = {'filename':'data/c2w3_lab2_data3.csv', 'label': '3 features', 'linestyle': 'dotted'}
file2 = {'filename':'data/c2w3_lab2_data2.csv', 'label': '2 features', 'linestyle': 'solid'}
files = [file1, file2]

# Treinar e plotar para cada conjunto de dados
utils.train_plot_diff_datasets(model, files, max_degree=4, baseline=250)

### Obter mais exemplos de treinamento

Por fim, você pode tentar minimizar o erro de validação cruzada obtendo mais exemplos.

Na célula abaixo, você treinará um modelo polinomial de 4º grau e, em seguida, traçará a *curva de aprendizado* do seu modelo para ver como os erros se comportam quando você obtém mais exemplos.

In [ ]:
# Preparar o conjunto de dados
x_train, y_train, x_cv, y_cv, x_test, y_test = utils.prepare_dataset('data/c2w3_lab2_data4.csv')
print(f"o formato do conjunto de treino (entrada) é: {x_train.shape}")
print(f"o formato do conjunto de treino (alvo) é: {y_train.shape}\n")
print(f"o formato do conjunto de validação cruzada (entrada) é: {x_cv.shape}")
print(f"o formato do conjunto de validação cruzada (alvo) é: {y_cv.shape}\n")

# Instanciar a classe do modelo
model = LinearRegression()

# Defina o grau do polinômio e treine o modelo usando subconjuntos do conjunto de dados.
utils.train_plot_learning_curve(model, x_train, y_train, x_cv, y_cv, degree= 4, baseline=250)

Os resultados mostram que o erro de validação cruzada começa a se aproximar do erro de treinamento à medida que você aumenta o tamanho do conjunto de dados. Outro insight que você pode obter com isso é que adicionar mais exemplos provavelmente não resolverá um problema de alta polarização. Isso ocorre porque o erro de treinamento permanece relativamente estável mesmo quando o conjunto de dados aumenta.

## Resumo

Neste laboratório, você pôde praticar como lidar com o alto viés e a alta variância no algoritmo de aprendizado.

Ao aprender a identificar esses problemas, você aprimorou sua intuição sobre o que tentar em seguida ao desenvolver seus modelos de aprendizado de máquina.